# テーブル定義をPythonコードで管理する

## 1. 背景と目的

Pythonを導入することでテーブル定義を一元的に管理することが可能と考えられる。ここでは、その可能性を探る。

一般に、テーブル定義の管理は煩雑なものとなりやすい。端的にはバージョン管理とドキュメントとの整合性が課題となる。
DDL(Create Table文)をレポジトリで一元管理する方針をとることもできるが、実行の手間とドキュメント生成をプログラミングする必要がある。これは、それなりに煩雑なものとなりやすい。

ここでは、Pythonと [SQLAlchemy](https://www.sqlalchemy.org/) での解決を図る。
SQLAlchemyは [Flask](https://msiz07-flask-docs-ja.readthedocs.io/ja/latest/) などでも採用されている、Pythonでは最も有名なORM(Object-Relational Mapping)である。
SQLAlchemyを利用する際は、DDLをPythonコードで表現し、環境(実際のRDBMS)への反映もPythonコードの実行で行うのが一般的である。

本項では、SQLAlchemyとTeradata Dialectを用いて、以下を実現する。

1. Python/SQLAlchemyTeradata DialectによってDDLを実装する
2. 1.をターゲットの環境に向けて実行することにより、実際にテーブルを作成する
3. 1.からテーブル定義ドキュメントを生成する

## 2. 方針

1. DDLは一つのPythonモジュールにまとめる
2. ターゲット環境への反映を関数として用意する
3. ドキュメント生成は、Json出力に留める
   - 実際にはHTMLドキュメントを生成することも可能だが、本来の目的からそれるため割愛する
   - Json出力されていれば、HTMLを生成することは容易である

## 3. 実装

### 3-1. DDLを記述する

In [1]:
# ddls.pyに必要テーブルがすべて記載されている。直接ファイルを開いて参照のこと
import ddls

### 3-2. DDLを実行してテーブルを作成する

#### DB接続情報を定義する。接続先はVMwareのTeradata Express

In [2]:
user = 'dbc'
password = 'dbc'
host = "192.168.11.9"
database = "example"
dbs_port = 1025

#### SQLAlchemyでTeradataに接続する

In [3]:
import sqlalchemy

engine = sqlalchemy.create_engine((
  f"teradatasql://{user}:{password}@{host}/?"
  f"&database={database}"
  f"&dbs_port={dbs_port}"
))

#### 初期化(Drop)した上で、作成(Create)する

In [4]:
ddls.drop_all(engine)
ddls.create_all(engine)

In [5]:
print(ddls.show_table(engine, 'iris'))

CREATE SET TABLE EXAMPLE.iris ,FALLBACK ,
     NO BEFORE JOURNAL,
     NO AFTER JOURNAL,
     CHECKSUM = DEFAULT,
     DEFAULT MERGEBLOCKRATIO,
     MAP = TD_MAP1
     (
      idx INTEGER NOT NULL,
      sepallength FLOAT NOT NULL,
      sepalwidth FLOAT NOT NULL,
      petallength FLOAT NOT NULL,
      petalwidth FLOAT NOT NULL,
      target VARCHAR(256) CHARACTER SET UNICODE NOT CASESPECIFIC NOT NULL)
UNIQUE PRIMARY INDEX ( idx );


### 3-3. テーブル定義ドキュメントを出力する

In [6]:
ddl_all_members = [getattr(ddls, name) for name in dir(ddls)]
tables = [member for member in ddl_all_members if isinstance(member, sqlalchemy.Table)]

In [7]:
def table_to_dict(table):
    dialect = None
    for item in table.kwargs.values():
        dialect = item
    return dict(
        name=table.name,
        columns=[column_to_dict(column) for column in table.columns],
        primary_index=dict(cols=dialect.opts['unique primary index'], unique=True) if 'unique primary index' in dialect.opts
        else dict(cols=dialect.opts['primary index'], unique=False)
    )

In [8]:
def column_to_dict(column):
    return dict(
        name=column.name,
        type=column.type,
        comment=column.comment,
        nullable=column.nullable,
    )

In [9]:
import pprint

for table in tables:
    print('----------')
    pprint.pprint(table_to_dict(table))

----------
{'columns': [{'comment': 'Unique ID',
              'name': 'idx',
              'nullable': False,
              'type': Integer()},
             {'comment': 'sepal length (cm)',
              'name': 'sepallength',
              'nullable': False,
              'type': Float()},
             {'comment': 'sepal width (cm)',
              'name': 'sepalwidth',
              'nullable': False,
              'type': Float()},
             {'comment': 'petal length (cm)',
              'name': 'petallength',
              'nullable': False,
              'type': Float()},
             {'comment': 'petal width (cm)',
              'name': 'petalwidth',
              'nullable': False,
              'type': Float()},
             {'comment': 'Target',
              'name': 'target',
              'nullable': False,
              'type': Unicode(length=256)}],
 'name': 'iris',
 'primary_index': {'cols': ['idx'], 'unique': True}}


あとは↑↑↑をHTMLやExcelとして出力すればOK